# To get started

- git clone https://github.com/bernardoleary/DevOpsDays-Wellington-2018.git
- cd DevOpsDays-Wellington-2018
- jupyter-notebook.exe

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Print Python version - just to make sure notebook is working
print (sys.version)

3.6.5 (v3.6.5:f59c0932b4, Mar 28 2018, 16:07:46) [MSC v.1900 32 bit (Intel)]


# Read in the CSV

In [2]:
# Read in the dataset
bug_predictor_sample_dataset = pd.read_csv('bug_predictor_sample_dataset.csv')

#bug_predictor_sample_dataset.dtypes
bug_predictor_sample_dataset[:3]

,merge_id,date_merged,requesting_user,no_of_commits,no_of_files,lines_of_code,bug_raised_in_uat,bug_raised_in_prod
0,30002389,8/08/2017,Michelle,2,3,45,0,0
1,30002396,14/02/2016,Raj,3,8,79,1,0
2,30002403,2/02/2014,Michelle,8,10,32,1,0


# Data formatting on date

In [3]:
# Add columns to put encoded date into
bug_predictor_sample_dataset['day_of_year'] = 0
bug_predictor_sample_dataset['week_of_year'] = 0
bug_predictor_sample_dataset['day_of_week'] = 0
bug_predictor_sample_dataset['quarter'] = 0
bug_predictor_sample_dataset['week'] = 0

# Change the date_merged datatype
bug_predictor_sample_dataset['date_merged'] = pd.to_datetime(bug_predictor_sample_dataset['date_merged'])

# Populate encoded date
for index, row in bug_predictor_sample_dataset.iterrows():
    bug_predictor_sample_dataset.at[index, 'day_of_year'] = row['date_merged'].dayofyear
    bug_predictor_sample_dataset.at[index, 'week_of_year'] = row['date_merged'].weekofyear
    bug_predictor_sample_dataset.at[index, 'day_of_week'] = row['date_merged'].dayofweek
    bug_predictor_sample_dataset.at[index, 'quarter'] = row['date_merged'].quarter
    bug_predictor_sample_dataset.at[index, 'week'] = row['date_merged'].week

# Print dataset    
bug_predictor_sample_dataset[:3]

,merge_id,date_merged,requesting_user,no_of_commits,no_of_files,lines_of_code,bug_raised_in_uat,bug_raised_in_prod,day_of_year,week_of_year,day_of_week,quarter,week
0,30002389,2017-08-08,Michelle,2,3,45,0,0,220,32,1,3,32
1,30002396,2016-02-14,Raj,3,8,79,1,0,45,6,6,1,6
2,30002403,2014-02-02,Michelle,8,10,32,1,0,33,5,6,1,5


# Data formatting on user

In [4]:
# Set up the requesting_user as a category so that we can encode
bug_predictor_sample_dataset["requesting_user"] = bug_predictor_sample_dataset["requesting_user"].astype('category')
bug_predictor_sample_dataset["requesting_user_cat"] = bug_predictor_sample_dataset["requesting_user"].cat.codes

# Print dataset
#bug_predictor_sample_dataset.dtypes
bug_predictor_sample_dataset[:3]

,merge_id,date_merged,requesting_user,no_of_commits,no_of_files,lines_of_code,bug_raised_in_uat,bug_raised_in_prod,day_of_year,week_of_year,day_of_week,quarter,week,requesting_user_cat
0,30002389,2017-08-08,Michelle,2,3,45,0,0,220,32,1,3,32,2
1,30002396,2016-02-14,Raj,3,8,79,1,0,45,6,6,1,6,4
2,30002403,2014-02-02,Michelle,8,10,32,1,0,33,5,6,1,5,2


# Build model

In [5]:
# Populate training set and target
X = bug_predictor_sample_dataset[['no_of_commits', 'no_of_files', 'lines_of_code', 'day_of_year', 'week_of_year', 'day_of_week', 'quarter', 'week', 'requesting_user_cat']].copy()
y = bug_predictor_sample_dataset[['bug_raised_in_uat']].copy()

# Train/test/split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Print model matrix shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# LinearRegression performs poorly by comparisson
# lm = LinearRegression()

lm = DecisionTreeClassifier() 
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)

# Print predictions
predictions[0:10]

(360, 9) (360, 1)
(90, 9) (90, 1)


array([1, 1, 1, 0, 1, 1, 0, 0, 0, 0], dtype=int64)

# Score the predeiction

In [6]:
# Print the scored model
print("Score:", model.score(X_test, y_test))

Score: 0.7666666666666667
